## Goal: Get ITI working on 1) BERT models and 2) LLaMa Models

In [ ]:
# Very fast installation of LLaMA weights via decapoda-research/llama-7b-hf
!pip install sentencepiece
!apt install git-lfs
!git lfs install
!git clone https://huggingface.co/decapoda-research/llama-7b-hf
!git clone https://huggingface.co/decapoda-research/llama-13b-hf

In [6]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer, LlamaForCausalLM
import dataclasses
import datasets
from IPython.display import HTML
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer, AutoTokenizer
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
from huggingface_hub import snapshot_download
import sentencepiece

# import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

torch.set_grad_enabled(False)

device = "cuda"

def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

ImportError: cannot import name 'LLaMATokenizer' from 'transformers' (/usr/local/lib/python3.8/dist-packages/transformers/__init__.py)

In [ ]:
## from local

from transformers import LlamaForCausalLM, LlamaTokenizer

# TODO
MODEL_PATH='llama_model/hf_llama_model'

tokenizer = LlamaTokenizer.from_pretrained(MODEL_PATH)
hf_model = LlamaForCausalLM.from_pretrained(MODEL_PATH, low_cpu_mem_usage = True)

In [2]:
# from huggingface

from huggingface_hub import snapshot_download
from accelerate import load_checkpoint_and_dispatch, init_empty_weights

checkpoint_location = snapshot_download("decapoda-research/llama-7b-hf")
checkpoint_location

Fetching 42 files:   0%|          | 0/42 [00:00<?, ?it/s]

'/root/.cache/huggingface/hub/models--decapoda-research--llama-7b-hf/snapshots/5f98eefcc80e437ef68d457ad7bf167c2c6a1348'

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = AutoModelForCausalLM.from_pretrained("decapoda-research/llama-7b-hf")

ValueError: Tokenizer class LLaMATokenizer does not exist or is not currently imported.

In [7]:
with init_empty_weights():
    model = LlamaForCausalLM.from_pretrained(checkpoint_location)
model = load_checkpoint_and_dispatch(
    model,
    checkpoint_location,
    device_map="auto",
    no_split_module_classes=["LlamaDecoderLayer"],
)

tok = LlamaTokenizer.from_pretrained(checkpoint_location)

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [9]:
torch.save(model.state_dict(), "llama.pt")

In [4]:
model = HookedTransformer.from_pretrained("llama-7b", hf_model=model, device=device, fold_ln=False, center_writing_weights=False, center_unembed=False)

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB (GPU 0; 22.19 GiB total capacity; 21.39 GiB already allocated; 122.50 MiB free; 21.40 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

model.tokenizer = tokenizer
model.tokenizer.add_special_tokens({"pad_token": "[PAD]"})

model.generate("The capital of Germany is", max_new_tokens=20, temperature=0)

outputs, cache = model.run_with_cache("The capital of Australia is Canberra, the country’s largest inland city. It is located in the Australian Capital Territory (ACT), which is a federal territory. The city is located on the banks of the Molonglo River, 280 km south-west of Sydney.\nThe city was founded in 1913 as a compromise between the six Australian colonies. The city was built to be the capital of the country, and it was named after the Aboriginal word for “meeting place”.\nThe city is the seat of the Australian Parliament, and the Parliament House is the most important building in the city. The Parliament House is a modern building, which was built in the 1980s. It is a circular building, which is surrounded by a lake.\nThe city is also the seat of the Australian Government. The city is the home of the High Court of Australia, the Supreme Court of the Australian Capital Territory, and the Australian Def")

In [ ]:
import einsum
import torch
import numpy as np

# Goal: define a function to take in hyperparameters Alpha and K, model probe values, and model activations to calculate the new activations
# Need to calculate alpha * sigma * theta
# theta is the truthful direction given by probe parameter
# sigma: standard deviation of head activation along truthful_dir (theta, either mass mean shift or probe weight direction)
# adapted from get_interventions_dict in Kenneth Li github

def get_act_std(head_activation, truthful_dir): # calculates standard deviations for one head
    """
    head_activation: (batch, d_model,)
    # truthful_dir: (d_model, )
    """
    truthful_dir /= torch.norm(truthful_dir, dim=-1, keepdim=True)
    proj_act = einops.einsum(head_activation, truthful_dir , "b d_m, d_m -> b d_m")
    return torch.std(proj_act, dim=0) # (d_m)

# truthful direction is difference in mean 
# returns (*, d_model)
def get_mass_mean_dir(all_activations, truth_indices): # 
    """
    all_activations: (batch, *, d_model)
    truth_indices: (batch, )
    """
    # print(f"shape of activations is {all_activations.shape}")
    # print(f"shape of truth_indices is {truth_indices.shape}")
    true_mass_mean = torch.mean(all_activations[truth_indices == 1], dim=0) #(*, d_model)
    false_mass_mean = torch.mean(all_activations[truth_indices == 0], dim=0)
    # (* d_model)

    return (true_mass_mean - false_mass_mean) / (true_mass_mean - false_mass_mean).norm()

# truthful direction is probe weight
# def get_probe_dirs(probe_list):
#     # probe is a list (n_heads len) of LogisticRegression objects
#     coefs = []
#     for probe in probe_list:
#         coefs.append(probe.coef_)
        
#     return torch.tensor(coefs, dtype=torch.float32, device=device)

def get_probe_dir(probe):
    probe_weights = torch.tensor(probe.coef_, dtype=torch.float32, device=device).squeeze()
    return probe_weights / probe_weights.norm(dim=-1)


# calculate the ITI addition (sigma * theta) for one head
# uses either MMD or probe
def calc_truth_proj(activation, use_MMD=False, use_probe=False, truth_indices=None, probe=None):
    '''
    activation is (batch, d_m)
    '''
    if use_MMD: # use mass mean direction -- average difference between true and false classified prompts (only one head)
        assert truth_indices is not None
        truthful_dir = get_mass_mean_dir(activation, truth_indices)
    else: # probe -- just the coefficients of the probe
        assert use_probe
        assert probe is not None
        truthful_dir = get_probe_dir(probe)

    # print(f"Old truthful dir direc is {truthful_dir.shape}")
    truthful_dir /= truthful_dir.norm(dim=-1)
    # print(f"New truthful dir direc is {truthful_dir.shape}")
    act_std = get_act_std(activation, truthful_dir)
    
    return einops.einsum(act_std, truthful_dir, "d_m, d_m -> d_m")

def patch_activation_hook_fn(activations, hook: HookPoint, head, old_activations, use_MMD=True, use_probe=False, truth_indices=None, probe=None):
    """
    activations: (batch, n_heads, d_model)
    hook: HookPoint
    term_to_add: (*, d_model)

    A hook that is meant to act on the "z" (output) of a given head, and add the "term_to_add" on top of it. Only meant to work a certain head. Will broadcast.
    """
    # print(f"in hook fn, old act shape is {old_activations.shape}")
    term_to_add = calc_truth_proj(old_activations[:,head], use_MMD, use_probe, truth_indices, probe)
    # print(f"v shape is {term_to_add.shape}")
    # print(f"activations shape is {activations.shape}")
    activations[:,-1,head] += term_to_add

# Calculates new_activations for topk and adds temporary hooks
def patch_top_activations(model, probe_accuracies, old_activations, topk=20, alpha=20, use_MMD=False, use_probe=False, truth_indices=None, probes=None):
    '''
    probe_accuracies: (n_layers, n_heads)
    old_activations: (batch, n_layers, n_heads, d_model)

    if use_probe is True, probes should be list in shape (n_layers, n_heads) filled with probes

    Goes into every single activation, and then tells it to add the ITI
    '''

    # print(f"old activations shape is {old_activations.shape}")

    top_head_indices = torch.topk(einops.rearrange(probe_accuracies, "n_l n_h -> (n_l n_h)"), k=topk).indices # take top k indices
    top_head_bools = torch.zeros(size=(probe_accuracies.shape[0] * probe_accuracies.shape[1],)) # set all the ones that aren't top to 0

    top_head_bools[top_head_indices] = torch.ones_like(top_head_bools[top_head_indices]) # set all the ones that are top to 1
    top_head_bools = einops.rearrange(top_head_bools, "(n_l n_h) -> n_l n_h", n_l=model.cfg.n_layers) # rearrange back
    
    for layer in range(probe_accuracies.shape[0]):
        for head in range(probe_accuracies.shape[1]):
            if top_head_bools[layer, head] == 1:

                if use_probe:
                    patch_activation_with_head = partial(patch_activation_hook_fn, head = head, old_activations = old_activations[:, layer], use_MMD=False, use_probe=use_probe, truth_indices=None, probe=probes[layer][head])
                else:
                    patch_activation_with_head = partial(patch_activation_hook_fn, head = head, old_activations = old_activations[:, layer], use_MMD=use_MMD, use_probe=False, truth_indices=truth_indices, probe=None)
                model.add_hook(utils.get_act_name("result", layer), patch_activation_with_head)